In [ ]:
from utils import *

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import pandas as pd

from ortools.sat.python import cp_model
from ortools.sat.python.cp_model import _SumArray

In [ ]:
values = getValues()
techs = getTechs()
tribes = getTribes()

In [ ]:
class GameState:
    turn = -1
    player = ""
    game = ""
    parent_state = None
    techs = {}
    previous_techs = {}
    vars = []
    tribe = ""
    cities = [] # 
    child_states = [] # GameStates
    raw_score = None
    full_score = None
    model = None
    solutions = None

    def __init__(self, turn, parent_state=None, previous_tech={}):
        self.turn = turn
        self.model = cp_model.CpModel()
        self.parent_state = parent_sate
        self.previous_tech = previous_tech

    def computeEndScore(self):
        self.

    def __repr__(self):
        return "GameState"
    
    def __str__(self):
        return "GameState turn %d, player %s" % (self.turn, self.player, self.game)

class Player:
    
    initialGameState = []

    def addScore(self):
        if len(initialGameState):
            pass
        pass

    def addTech(self):
        pass

    def addTribe(self, tribe):
        data = getTribe()[tribe]
        pass

In [ ]:
g = GameState()


In [ ]:
def solve(raw_score, full_score):
    model = cp_model.CpModel()
    eq_dict, sym_dict = addAllVars(model, values)

    buildScoreConstraint(model, eq_dict, full_score, True)
    buildScoreConstraint(model, eq_dict, raw_score, False)
    model.Add(_SumArray([sym_dict[e] for e in values['units'].keys()]) < 20)


    solver = cp_model.CpSolver()
    solution_printer = DataFrameSolutionAggregator(sym_dict.values())
    status = solver.SearchForAllSolutions(model, solution_printer)
    return solution_printer

In [ ]:
# superb_ruin
scores = [
    ([515, 515], []),
    ([565, 565], [440, 440]),
    ([590, 600], [450, 450]),
    ([855, 635], [700, 460]),
    ([1385, 670], [1230, 460]),
    ([1860, 1215], [1700, 990]),
    ([2230, 1815], [2010, 1530])
]

In [ ]:
player_scores = [
    [565, 440],
    [590, 450],
    [855, 700],
    [1385, 1230],
    [1860, 1700],
    [2230, 2010]
]

In [ ]:
player_2_scores = [
    [565, 440],
    [600, 450],
    [635, 460],
    [670, 460],
    [1215, 990],
    [1815, 1530]
]

In [ ]:
def divide_scores(scores):


    return selected_scores + divide_score(scores[1:])

In [ ]:
for i in range(len(player_scores) - 1):
    raw_delta = player_scores[i + 1][1] - player_scores[i][1]
    full_delta = player_scores[i + 1][0] - player_scores[i][0]
    print(full_delta, raw_delta)
    # sp = solve(raw_score=raw_delta, full_score=full_delta)
    # print(sp.get_data_frame().to_string())

In [ ]:
print(sp.get_data_frame())

# Equations: 

full score = score(units) + score(tech) + score(city) + score(vision)

raw score =  score(units) + score(tech) + score(city)

'remaining stars + spt + extra stars = stars(units) + stars(tech) + star(building)'

In [ ]:
import re
from utils import * 
model = cp_model.CpModel()

past_turn = None
dictionnaries = {}
scores = {
    "start": [515, 390],
    "t0": [565, 440],
    "t1": [600, 450],
    "t2": [635, 460],
    "t3": [670, 460],
    "t4": [1215, 990],
    "t5": [1815, 1530]
}

for turn in ["start", "t0"]:#, "t1", "t2", "t3"]:#, "t4", "t5"]:
    dictionnaries[turn] = buildDictionnaries(model, turn)
    full_score = scores[turn][0] 
    raw_score = scores[turn][1]

    buildAllConstraints(model, turn, dictionnaries)

    setRawScore(model, turn, dictionnaries[turn]['scores'], raw_score)
    setFullScore(model, turn, dictionnaries[turn]['scores'], full_score)
    setTribe(model, turn, "Bardur", dictionnaries[turn]['tribes'])

    linkTurns(model, turn, dictionnaries)
    addInitialState(model, turn, dictionnaries)
    past_turn = turn

addSolution(model, turn, dictionnaries)

solver = cp_model.CpSolver()
output_variables = list(h for turn, turn_dict in dictionnaries.items() for topic, topic_dict in turn_dict.items() for h in topic_dict.values() if topic != "equations")
solution_printer = DataFrameSolutionAggregator(output_variables)
status = solver.SearchForAllSolutions(model, solution_printer)
print('Solve status: %s' % solver.StatusName(status))

In [ ]:
x = solution_printer.get_data_frame()

In [ ]:
printValueList(x)

In [ ]:
x[x['t2_population'] ==]

In [ ]:
cleaned = chooseBestSolution(x[x['t2_population'] == 2])

In [ ]:
cleaned

In [ ]:
def printScenario(full_df):
    regex = getScenarioRegex()
    selected_columns = [u for u in full_df.columns if sum(re.search(r, u) is not None for r in regex) != 0]
    #print(selected_columns)
    df = full_df[selected_columns]
    identical_columns = list(filter(lambda x: len(df[x].unique()) == 1, selected_columns))
    different_columns = list(filter(lambda x: len(df[x].unique()) != 1, selected_columns))
    turns = list(set(map(lambda u: u.split("_")[0], selected_columns)))
    for turn_i in sorted(turns):
        turn_string = ("Turn " + turn_i[1:] + ":" if turn_i.startswith("t") else "Start:")
        print(turn_string)
        print("  Common:")
        for element in identical_columns:
            # print(df[element].iloc[0])
            if element.startswith(turn_i) and df[element].iloc[0] != 0:
                print("   " + " ".join(element.split("_")[1:]) + ": " + str(df[element].iloc[0]))
        if sum(e.startswith(turn_i) for e in different_columns) > 0:
            for i, row in df.iterrows():
                print(" Scenario " + str(i) + ":")
                print("  Specific:")
                for element in different_columns:
                    if element.startswith(turn_i):
                        print("   " + " ".join(element.split("_")[1:]) + ": " + str(row[element]))
printScenario(cleaned)

In [ ]:
variants = printVariants(cleaned)

In [ ]:
x['start_warrior'].head(1)

In [ ]:
variants

In [ ]:
# def prettyPrint(variants):
#     turns = list(set(map(lambda u: u[0].split("_")[0], variants)))
#     for turn_i in turns:
#         turn_string = ("Turn " + turn_i[1:] + ":" if turn_i.startswith("t") else "Start:")
#         print(turn_string)
#         for element, value in map(lambda u: (" ".join(u[0].split("_")[1:]), u[1]), filter(lambda u: u[0].startswith(turn_i), # variants)):
#             print(element + ":", ", ".join(map(lambda u: str(u), value)))
# prettyPrint(variants)

In [ ]:
printValueList(cleaned)

In [ ]:
printValueList(x)

In [ ]:
{
		"name": "water_temple",
		"cost": 20,
		"pop": 1,
		"requires": "acquatism"
	},{
		"name": "forest_temple",
		"cost": 20,
		"pop": 1,
		"requires": "spiritualism"
	},{
		"name": "mountain_temple",
		"cost": 20,
		"pop": 1,
		"requires": "meditation"
	},{
		"name": "temple",
		"cost": 20,
		"pop": 1,
		"requires": "free spirit"
	},{
		"name": "road_conection",
		"cost": 0,
		"pop": 2,
		"requires": "roads"
	},{
		"name": "water_connection",
		"cost": 0,
		"pop": 2,
		"requires": "sailing"
	}